In [1]:
import numpy as np
import random
import pandas as pd
import os
from PIL import Image
import matplotlib.pyplot as plt
from scipy.linalg import sqrtm
#from skimage.transform import resize
import cv2
from keras.preprocessing.image import load_img, img_to_array, array_to_img
from keras.layers import Activation, Dense, Input, Dropout, BatchNormalization
from keras.layers import Conv2D, Conv2DTranspose, Input, Flatten, Dense, Reshape, Concatenate, Lambda, Layer
from keras.layers.merge import concatenate
from keras.models import Model
from keras.optimizers import Adam
from keras.losses import mean_squared_error, binary_crossentropy
from keras import metrics
from keras.applications.inception_v3 import preprocess_input, InceptionV3
import keras.backend as K

Using TensorFlow backend.


In [26]:
## Constants
OLD_IMAGE_DIMS = (218, 178, 3)
NEW_IMAGE_DIMS = (64, 64, 3)
CROP_IMAGE_DIMS = (25, 45, 153, 173)
BATCH_SIZE = 128 # Hva er dette?
N = 6
NUM_ATTRIBUTES = 40
LATENT_DIM = 64
TOT_IMAGES = 202599
EPOCHS = 1

In [27]:
## Constants needed to run in Google Colab
PATH = './'
IMAGES = 'img_align_celeba/'
ATTRIBUTES = 'list_attr_celeba.txt'

In [28]:
## Loading dataset
def get_attributes(filename):
  f = open(filename, 'r')
  lines = f.readlines()
  attributes = []
  i = 0
  for line in lines:
      if i != 0 and i != 1:
        line = line.split()
        #line.pop(0)
        attributes.append(line)
      i += 1
  return attributes

def initialize_training_set(): # N or batch size??
    all_attributes = get_attributes(PATH + ATTRIBUTES)
    chosen_info = random.sample(all_attributes, N)
    chosen_attributes = [info[1:] for info in chosen_info]
    chosen_images = [info[0] for info in chosen_info]
    resized_images = []
    
    while True:
        for ind in range(len(chosen_info)):
            image_path = PATH + IMAGES + chosen_images[0]
            #img = cv2.imread(np.frombuffer(image_path), cv2.IMREAD_COLOR)#[...,::-1] / 255.0
            img = cv2.imread(image_path, cv2.IMREAD_COLOR)#[...,::-1] / 255.0
            img = img[45:173 , 25:153]
            img = cv2.resize(src=img, dsize=(64, 64))
            #img = Image.open(image_path).crop(CROP_IMAGE_DIMS)
            #img = img.resize((NEW_IMAGE_DIMS[0], NEW_IMAGE_DIMS[1]))
            #img = cv2.resize(src=img, dsize=[NEW_IMAGE_DIMS[0], NEW_IMAGE_DIMS[0]])

            #image = cv2.imdecode(np.frombuffer(image_path, np.uint8), cv2.IMREAD_COLOR)[...,::-1] / 255.0
            #image = image[45:173 , 25:153]
            #image = cv2.resize(src=image, dsize=(Image_Dim_Crop[0], Image_Dim_Crop[1]))
            #resized_images += [img]
            np.append(resized_images, img)

            """
            for name in chosen_images:
                img = np.array((Image.open(PATH + IMAGES + name).crop(CROP_IMAGE_DIMS)).resize((NEW_IMAGE_DIMS[0], NEW_IMAGE_DIMS[1])))
                resized_images.append(img)
            resized_images = np.array(resized_images)
            resized_images = np.array([imgs.astype('float32') for imgs in resized_images])
            """
        resized_images = np.array(resized_images, dtype=np.float32)
        chosen_attributes = np.array(chosen_attributes, dtype=np.float32)

        yield [resized_images, chosen_attributes], None

training_set = initialize_training_set()
print(training_set)

<generator object initialize_training_set at 0x13de146d0>


In [ ]:
"""
attribute_folder = PATH + ATTRIBUTES

def celeba_generator():
  read_attributes = pd.read_csv(attribute_folder, skiprows=2,header=None,delim_whitespace=True)
  #attributes = np.array(pd.read_csv(Library + Attributes_txt_file, skiprows=2, header=None, delim_whitespace=True))
  attribute_array = np.array(read_attributes)
  with ZipFile(Celeba_Folder, 'r') as Celeba_Dataset: #Dundrer gjennom bildene i Celeba
    while True: #Trekker ut random bilder fra 1D attributt array innenfor batch sizen
      images = attribute_array[np.random.choice(attribute_array.shape[0], Batch_Size, replace=False)] #random.choice generates a random sample from 1D array
      image_batch = []
      attributes_batch = []
      for image_info in images: #Litt usikker på hva som skjer her?
        image = cv2.imdecode(np.frombuffer(Celeba_Dataset.read(Celeba_images + image_info[0]), np.uint8), cv2.IMREAD_COLOR)[...,::-1] / 255.0
        image = image[45:173 , 25:153]
        image = cv2.resize(src=image, dsize=(Image_Dim_Crop[0], Image_Dim_Crop[1]))
        image_batch += [image]
        attributes_batch += [image_info[1:]]
      image_batch = np.array(image_batch, dtype=np.float32)
      attribute_batch = np.array(attributes_batch, dtype=np.int32)

      yield [image_batch, attribute_batch], None

training_set = celeba_generator()

print(training_set)
"""


In [29]:

class Sampling(Layer):
  def call(self, inputs):
    z_mean, z_log_var = inputs
    batch = K.shape(z_mean)[0]
    dim = K.shape(z_mean)[1]
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

def VAE():
  #Define encoder model.
  input_img = Input(shape = NEW_IMAGE_DIMS, name='input_img')
  labels = Input(shape = (NUM_ATTRIBUTES,), name='labels')

  x = Conv2D(filters = 32, kernel_size = 3, strides = 2, padding = 'same', activation = 'relu')(input_img)
  x = Conv2D(filters = 64, kernel_size = 3, strides = 2, padding = 'same', activation = 'relu')(x)
  x = Conv2D(filters = 128, kernel_size = 3, strides = 2, padding = 'same', activation = 'relu')(x)
  x = Conv2D(filters = 256, kernel_size = 3, strides = 2, padding = 'same', activation = 'relu')(x)
  x = Conv2D(filters = 512, kernel_size = 3, strides = 2, padding = 'same', activation = 'relu')(x)

  #encode = Encode_Decode(input_img) #Done

  shape_before_flattening = K.int_shape(x)[1:]
  #shape_before_flattening = K.int_shape(encode)[1:]

  x = Flatten()(x)

  z_mean = Dense(LATENT_DIM, name='z_mean')(x)
  z_log_sigma = Dense(LATENT_DIM, name='z_log_sigma')(x)
  z = Sampling()([z_mean, z_log_sigma])

  zy = Concatenate()([z, labels])

  inputs_embedding = Input(shape=(LATENT_DIM + NUM_ATTRIBUTES,))
  embedding = Dense(np.prod(shape_before_flattening))(inputs_embedding)
  embedding = Reshape(shape_before_flattening)(embedding)

  #Decoding
  x_ = Conv2DTranspose(filters = 256, kernel_size = 3, strides = 2,  padding = 'same', activation = 'relu')(embedding)
  x_ = Conv2DTranspose(filters = 128, kernel_size = 3, strides = 2,  padding = 'same', activation = 'relu')(x_)
  x_ = Conv2DTranspose(filters = 64, kernel_size = 3, strides = 2,  padding = 'same', activation = 'relu')(x_)
  x_ = Conv2DTranspose(filters = 32, kernel_size = 3, strides = 2,  padding = 'same', activation = 'relu')(x_)
  x_ = Conv2DTranspose(filters = 3, kernel_size = 3, strides = 2,  padding = 'same', activation = 'sigmoid')(x_)

  #x_hat = Encode_Decode(embedding)

  encoder = Model(inputs = [input_img, labels], outputs = zy, name="encoder")
  decoder = Model(inputs = inputs_embedding, outputs = x_, name="decoder")

  vae_out = decoder(encoder([input_img, labels]))

  rec_loss =  np.prod(NEW_IMAGE_DIMS) * binary_crossentropy(Flatten()(input_img), Flatten()(vae_out))
  kl_loss = - 0.5 * K.sum(1 + z_log_sigma - K.square(z_mean) - K.exp(z_log_sigma), axis=-1)
  vae_loss = K.mean(rec_loss + kl_loss)

  vae = Model(inputs = [input_img, labels], outputs = vae_out, name="vae")

  vae.add_loss(vae_loss)

  optimizer = Adam(lr=0.0005, beta_1 = 0.5)
  vae.compile(optimizer)

  return vae, encoder, decoder

vae, encoder, decoder = VAE()

encoder.summary()
decoder.summary()



Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_img (InputLayer)          (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
conv2d_36 (Conv2D)              (None, 32, 32, 32)   896         input_img[0][0]                  
__________________________________________________________________________________________________
conv2d_37 (Conv2D)              (None, 16, 16, 64)   18496       conv2d_36[0][0]                  
__________________________________________________________________________________________________
conv2d_38 (Conv2D)              (None, 8, 8, 128)    73856       conv2d_37[0][0]                  
____________________________________________________________________________________________

/usr/local/lib/python3.7/site-packages/keras/engine/training_utils.py:819: UserWarning: Output decoder missing from loss dictionary. We assume this was done on purpose. The fit and evaluate APIs will not be expecting any data to be passed to decoder.
  'be expecting any data to be passed to {0}.'.format(name))


In [31]:
vae.fit(training_set, steps_per_epoch=TOT_IMAGES//BATCH_SIZE, verbose=1, epochs=EPOCHS)

Epoch 1/1
1582/1582 [==============================] - 383s 242ms/step - loss: nan


In [32]:
# Save model weights
vae.save_weights('./vae.h5')
decoder.save_weights('./decoder.h5')
encoder.save_weights('./encoder.h5')

In [37]:
def scale_images(images, new_shape):
    images_list = list()
    for image in images:
        # resize with nearest neighbor interpolation
        new_image = resize(image, new_shape, order=1)
        # store
        images_list.append(new_image)
    return np.asarray(images_list)

def calculate_fid(act1 , act2):
    mu1, sigma1 = act1.mean(axis=0), np.cov(act1, rowvar=False)
    mu2, sigma2 = act2.mean(axis=0), np.cov(act2, rowvar=False)
    ssdiff = np.sum((mu1 - mu2)**2.0)
    covmean = sqrtm(sigma1.dot(sigma2))
    if np.iscomplexobj(covmean):
        covmean = covmean.real
    fid = ssdiff + np.trace(sigma1 + sigma2 - 2.0 * covmean)
    return fid

inception_model = InceptionV3(include_top=False, pooling='avg')

87916544/87910968 [==============================] - 204s 2us/step


In [39]:
def celeba_vae_eval_generator():
  attributes = np.array(pd.read_csv(Library + Attributes_txt_file, skiprows=2, header=None, delim_whitespace=True))
  attributes = attributes[np.random.choice(attributes.shape[0], 10000, replace=False)]
  with ZipFile(Library + Celeba_Zip, 'r') as Celeba_Dataset:
    batched = 0
    while batched < 10000:
      images = attributes[batched: batched + Batch_Size]
      image_batch = []
      attributes_batch = []
      for image_info in images:
        image = cv2.imdecode(np.frombuffer(Celeba_Dataset.read(Celeba_images + image_info[0]), np.uint8), cv2.IMREAD_COLOR)[...,::-1] / 255.0
        image = image [45:173 , 25:153]
        image = cv2.resize(src=image, dsize=(Image_Dim_Crop[0], Image_Dim_Crop[1]))
        image_batch += [image]
        attributes_batch += [image_info[1:]]
      image_batch = np.array(image_batch, dtype=np.float32)
      attribute_batch = np.array(attributes_batch, dtype=np.int32)

      batched += Batch_Size

      yield [image_batch, attribute_batch]

validation_dataset = celeba_vae_eval_generator()

In [40]:
print("Calculating activations for each batch...")
img_act = np.array([]).reshape((0, 2048))
pred_act = np.array([]).reshape((0, 2048))
i = 1
for batch in validation_dataset:
  images = batch[0]
  print("Batch", i, "- Num. samples:", images.shape[0])
  predictions = vae.predict(batch)
  if i == 1:
    imgs1 = images
    prediction1 = predictions
  images = images * 255.0
  predictions = predictions * 255.0
  images1 = scale_images(images, (299, 299, 3))
  images1 = preprocess_input(images1)
  images2 = scale_images(predictions, (299, 299, 3))
  images2 = preprocess_input(images2)
  a = inception_model.predict(images1)
  b = inception_model.predict(images2)
  img_act = np.concatenate((img_act, a), axis = 0)
  pred_act = np.concatenate((pred_act, b), axis = 0)
  i += 1
print("Done!")
show_batch(imgs1)
show_batch(prediction1)

Calculating activations for each batch...


NameError: name 'Library' is not defined